# PGA Tour Web Scraping

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
#from selenium import webdriver
from bs4 import BeautifulSoup
import os
import lxml.html as lh
import requests as req
import urllib
import json
from itertools import chain
import sqlite3 as db

We will scrape data from the PGA Tour website using their api. The api url can be found by inspecting the tables we want (from https://www.pgatour.com/players/player.28237.rory-mcilroy.html for Rory McIlroy) and using the Network tab to view from where the website collects it's data.

In [2]:
url = 'https://statdata-api-prod.pgatour.com/api/clientfile/YtdPlayerStatsArchive?P_ID=28237&YEAR=2021&format=json&userTrackingId=exp=1628181998~acl=*~hmac=0352b02b9bab73b560ec308974d8b77130caf5cf2e3f1cabdacb7d49510c575a'

In [3]:
api_id = {'McIlroy' : '28237'}
player_id = api_id['McIlroy']
year = ''
hmac = ''
tracking_id = ''
url_format = 'https://statdata-api-prod.pgatour.com/api/clientfile/YtdPlayerStatsArchive?P_ID=' + player_id + '&YEAR=' + year + '&format=json&userTrackingId=exp=' + tracking_id + '~acl=*~hmac=' + hmac

The player_id and year are easy to choose. But the hmac and tracking_id change every time I open the webpage. I need to find a way to automate the retrieval of the hmac and tracking_id for each webpage.

Note: this url has to be updated frequently. Potentially every time you run this file you may need to update the url

In [4]:
request = req.get(url)
j = request.json()

In [5]:
j

{'header': {'version': '1.0'},
 'plrs': [{'plrNum': '28237',
   'plrName': 'Rory McIlroy',
   'years': [{'year': '2021',
     'disclaimer': 'Player did not meet the minimum number of rounds requirement to be officially ranked. The rank indicated is the position the player would be ranked if the minimum had been met.',
     'tours': [{'tourCodeLC': 'r',
       'tourCodeUC': 'R',
       'tourName': 'PGA TOUR',
       'minRndsMet': 'Y',
       'year': '2021',
       'lastTrnProc': "Olympic Men's Golf Competition",
       'trnNum': '470',
       'permNum': '519',
       'statCats': [{'catName': 'Recap',
         'stats': [{'statID': '02567',
           'name': 'SG: Off-the-Tee',
           'value': '.636',
           'rank': '8',
           'projRank': '',
           'additionals': [{'title': 'Total SG:OTT', 'value': '29.912'},
            {'title': 'Measured Rounds', 'value': '47'}]},
          {'statID': '02568',
           'name': 'SG: Approach the Green',
           'value': '.501',
  

In [6]:
recap_data = j.get('plrs')[0]#.get('years')[0].get('tours')[0].get('statCats')[0].get('stats')

In [7]:
off_the_tee_data = j.get('plrs')[0].get('years')[0].get('tours')[0].get('statCats')[1].get('stats')

In [8]:
app_the_green_data = j.get('plrs')[0].get('years')[0].get('tours')[0].get('statCats')[2].get('stats')

In [9]:
ar_the_green_data = j.get('plrs')[0].get('years')[0].get('tours')[0].get('statCats')[3].get('stats')

In [10]:
putting_data = j.get('plrs')[0].get('years')[0].get('tours')[0].get('statCats')[4].get('stats')

In [11]:
scoring_data = j.get('plrs')[0].get('years')[0].get('tours')[0].get('statCats')[5].get('stats')

In [12]:
streaks_data = j.get('plrs')[0].get('years')[0].get('tours')[0].get('statCats')[6].get('stats')

In [13]:
recap = pd.DataFrame(recap_data)

In [14]:
recap['years']

0    {'year': '2021', 'disclaimer': 'Player did not...
Name: years, dtype: object

We nearly have everything we need, our problem is in the additionals column. Notice how our stats query is a list of dictionaries with embedded dictionaries for the 'additionals' key. The other columns converted to the data frame nicely because they did not have embedded dictionaries within them. 

In [15]:
#recap_additionals = pd.DataFrame(list(chain.from_iterable(recap['additionals'])))

Now we have a separate data frame for the additional statistics column from our recap data frame. Now we can do this for all of our categories.

In [16]:
off_the_tee = pd.DataFrame(off_the_tee_data)

In [17]:
off_the_tee

,statID,name,value,rank,projRank,additionals
0,02674,SG: Tee-to-Green,1.293,11,,"[{'title': 'SG:OTT', 'value': '.636'}, {'title..."
1,02567,SG: Off-the-Tee,.636,8,,"[{'title': 'Total SG:OTT', 'value': '29.912'},..."
2,101,Driving Distance,317.7,2,,"[{'title': 'Total Distance', 'value': '36,850'..."
3,317,Driving Distance - All Drives,308.4,2,,"[{'title': 'Total Distance', 'value': '201,705..."
4,159,Longest Drives,379,83,,"[{'title': 'Tournament', 'value': 'the Memoria..."
5,102,Driving Accuracy Percentage,55.32%,177,,"[{'title': 'Fairways Hit', 'value': '447'}, {'..."
6,02420,Distance from Edge of Fairway,30' 10,167,,"[{'title': 'Total Distance (Feet)', 'value': '..."
7,459,Left Rough Tendency,14.81%,163,,"[{'title': 'Total Left rough', 'value': '96'},..."
8,460,Right Rough Tendency,17.28%,170,,"[{'title': 'Total Right rough', 'value': '112'..."
9,129,Total Driving,179,55,,"[{'title': 'Distance Rank', 'value': '2'}, {'t..."


In [18]:
off_the_tee_additionals = pd.DataFrame(list(chain.from_iterable(off_the_tee['additionals'])))

In [19]:
app_the_green = pd.DataFrame(app_the_green_data)

In [20]:
ar_the_green = pd.DataFrame(ar_the_green_data)

In [21]:
putting = pd.DataFrame(putting_data)

In [22]:
scoring = pd.DataFrame(scoring_data)

In [23]:
streaks = pd.DataFrame(streaks_data)

In [24]:
app_the_green_additionals = pd.DataFrame(list(chain.from_iterable(app_the_green['additionals'])))

In [25]:
ar_the_green_additionals = pd.DataFrame(list(chain.from_iterable(ar_the_green['additionals'])))

In [26]:
putting_additionals = pd.DataFrame(list(chain.from_iterable(putting['additionals'])))

In [27]:
scoring_additionals = pd.DataFrame(list(chain.from_iterable(scoring['additionals'])))

In [28]:
streaks_additionals = pd.DataFrame(list(chain.from_iterable(streaks['additionals'])))

In future iterations I will make a function that automates this. But for now this will suffice.

Let's now concatenate the dataframes, drop the additionals column from the primary data, and drop the projRank column. Let's also remove duplicate rows from both.

In [29]:
McIlroy_data = pd.concat([recap, off_the_tee, app_the_green, ar_the_green, putting, scoring, streaks], axis = 0)
McIlroy_data = McIlroy_data.drop(['projRank', 'additionals'], axis = 1)
McIlroy_data = McIlroy_data.drop_duplicates('statID')

In [30]:
McIlroy_additionals = pd.concat([off_the_tee_additionals, #recap_additionals
                                 app_the_green_additionals, ar_the_green_additionals,
                                 putting_additionals, scoring_additionals,
                                 streaks_additionals], axis = 0)
McIlroy_additionals = McIlroy_additionals.drop_duplicates('title')

In [31]:
McIlroy_data_transposed = McIlroy_data.transpose()
#McIlroy_data_transposed.rename(columns = McIlroy_data_transposed[2,:])
McIlroy_data_transposed.columns = McIlroy_data_transposed.iloc[1, :]
McIlroy_data_transposed

plrName,Rory McIlroy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
plrNum,28237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
plrName,Rory McIlroy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
years,"{'year': '2021', 'disclaimer': 'Player did not...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
statID,NaN,02674,02567,101,317,159,102,02420,459,460,...,295,294,483,296,297,298,452,449,02672,02673
name,NaN,SG: Tee-to-Green,SG: Off-the-Tee,Driving Distance,Driving Distance - All Drives,Longest Drives,Driving Accuracy Percentage,Distance from Edge of Fairway,Left Rough Tendency,Right Rough Tendency,...,Best YTD 1-Putt or Better Streak,Best YTD Streak w/o a 3-Putt,Current Streak without a 3-Putt,Consecutive Sand Saves,Consecutive Fairways Hit,Consecutive GIR,Consecutive Holes Below Par,Consecutive Par 3 Birdies,Consecutive Birdies Streak,Consecutive Birdies/Eagles streak
value,NaN,1.293,.636,317.7,308.4,379,55.32%,30' 10,14.81%,17.28%,...,9,125,37,8,10,15,5,3,5,2
rank,NaN,11,8,2,2,83,177,167,163,170,...,29,146,126,23,252,141,16,14,15,90


In [32]:
McIlroy_additionals_transposed = McIlroy_additionals.transpose()
McIlroy_additionals_transposed.columns = McIlroy_additionals_transposed.iloc[0,:]
McIlroy_additionals_transposed


title,SG:OTT,Measured Rounds,Total SG:OTT,Total Distance,Total Drives,# of Drives,Tournament,Round,Fairways Hit,Possible Fairways,...,Total Late Rounds,Par 3 Birdies or Better,Par 3 Holes,Par 4 Birdies or Better,Par 4 Holes,Par 5 Birdies or Better,Par 5 Holes,Total Birdies,Start Tournament ID,Current Streak
title,SG:OTT,Measured Rounds,Total SG:OTT,Total Distance,Total Drives,# of Drives,Tournament,Round,Fairways Hit,Possible Fairways,...,Total Late Rounds,Par 3 Birdies or Better,Par 3 Holes,Par 4 Birdies or Better,Par 4 Holes,Par 5 Birdies or Better,Par 5 Holes,Total Birdies,Start Tournament ID,Current Streak
value,.636,47,29.912,"36,850",116,654,the Memorial,3,447,808,...,31,33,236,111,602,110,206,254,"2,021,340",37


In [33]:
McIlroy_data.loc[McIlroy_data['statID'] == '103']

,plrNum,plrName,years,statID,name,value,rank
1,NaN,NaN,NaN,103,Greens in Regulation Percentage,65.42%,110


In [34]:
stat_ids = McIlroy_data['statID'].values
stat_ids

array([nan, '02674', '02567', '101', '317', '159', '102', '02420', '459',
       '460', '129', '02401', '02412', '02568', '103', '419', '486',
       '02357', '331', '437', '432', '433', '431', '02361', '02360',
       '02359', '02358', '076', '075', '074', '340', '339', '338', '337',
       '336', '02374', '02373', '02372', '02371', '02370', '02369',
       '02368', '02367', '02366', '02365', '02364', '02363', '02362',
       '350', '351', '02569', '111', '375', '130', '364', '363', '366',
       '367', '368', '369', '02564', '02428', '104', '402', '413', '426',
       '119', '393', '394', '395', '396', '341', '342', '343', '344',
       '345', '346', '347', '348', '485', '484', '405', '406', '407',
       '408', '498', '349', '438', '115', '02675', '299', '120', '108',
       '156', '107', '106', '155', '105', '148', '149', '117', '285',
       '118', '207', '208', '292', '293', '142', '143', '144', '219',
       '112', '113', '114', '352', '160', '122', '482', '295', '294',
       '

In [35]:
stat_names = McIlroy_data['name'].values

In [36]:
stats = {stat_ids[i] : stat_names[i] for i in range(1, len(stat_ids))}

In [37]:
stats

{'02674': 'SG: Tee-to-Green',
 '02567': 'SG: Off-the-Tee',
 '101': 'Driving Distance',
 '317': 'Driving Distance - All Drives',
 '159': 'Longest Drives',
 '102': 'Driving Accuracy Percentage',
 '02420': 'Distance from Edge of Fairway',
 '459': 'Left Rough Tendency',
 '460': 'Right Rough Tendency',
 '129': 'Total Driving',
 '02401': 'Club Head Speed',
 '02412': 'Total Driving Efficiency',
 '02568': 'SG: Approach the Green',
 '103': 'Greens in Regulation Percentage',
 '419': 'Going for the Green',
 '486': 'Going for the Green - Hit Green Pct.',
 '02357': 'Going for the Green - Birdie or Better',
 '331': 'Proximity to Hole',
 '437': 'Rough Proximity',
 '432': 'Left Rough Proximity',
 '433': 'Right Rough Proximity',
 '431': 'Fairway Proximity',
 '02361': 'Approaches from &gt; 275 yards',
 '02360': 'Approaches from 250-275 yards',
 '02359': 'Approaches from 225-250 yards',
 '02358': 'Approaches from 200-225 yards',
 '076': 'Approaches from 50-75 yards',
 '075': 'Approaches from 75-100 yards

In [38]:
data = {'stat_ids' : stat_ids[1:], 'stat_names': stat_names[1:]}
stat_ids_df = pd.DataFrame.from_dict(data)
conn = db.connect('stat_ids.db')
stat_ids_df.to_sql('stat_ids', conn, if_exists = 'replace')
conn.close()

Now we can use this dictionary to construct dataframes for each statistic.

In [39]:
def get_links(stats, year):
    """
    Create a list of links for each statistic in a specified year.
    """
    base_url = 'https://www.pgatour.com/content/pgatour/stats/stat.'
    urls = [base_url + str(id) + '.y' + str(year) + '.html' for id in stats]
    return urls

Let's just use one of these urls for now to see what they look like under the hood

In [40]:
conn = db.connect('stat_ids.db')

c = conn.cursor()

query = conn.execute("SELECT * From stat_ids")
cols = [column[0] for column in query.description]
stat_ids = pd.DataFrame.from_records(data = query.fetchall(), columns = cols)

conn.close()

In [41]:
stat_ids

,index,stat_ids,stat_names
0,0,02674,SG: Tee-to-Green
1,1,02567,SG: Off-the-Tee
2,2,101,Driving Distance
3,3,317,Driving Distance - All Drives
4,4,159,Longest Drives
...,...,...,...
122,122,298,Consecutive GIR
123,123,452,Consecutive Holes Below Par
124,124,449,Consecutive Par 3 Birdies
125,125,02672,Consecutive Birdies Streak


In [42]:
stat_nums = stat_ids['stat_ids']
stat_names = stat_ids['stat_names']
stat_dict = dict(zip(stat_nums, stat_names))
stat_dict

{'02674': 'SG: Tee-to-Green',
 '02567': 'SG: Off-the-Tee',
 '101': 'Driving Distance',
 '317': 'Driving Distance - All Drives',
 '159': 'Longest Drives',
 '102': 'Driving Accuracy Percentage',
 '02420': 'Distance from Edge of Fairway',
 '459': 'Left Rough Tendency',
 '460': 'Right Rough Tendency',
 '129': 'Total Driving',
 '02401': 'Club Head Speed',
 '02412': 'Total Driving Efficiency',
 '02568': 'SG: Approach the Green',
 '103': 'Greens in Regulation Percentage',
 '419': 'Going for the Green',
 '486': 'Going for the Green - Hit Green Pct.',
 '02357': 'Going for the Green - Birdie or Better',
 '331': 'Proximity to Hole',
 '437': 'Rough Proximity',
 '432': 'Left Rough Proximity',
 '433': 'Right Rough Proximity',
 '431': 'Fairway Proximity',
 '02361': 'Approaches from &gt; 275 yards',
 '02360': 'Approaches from 250-275 yards',
 '02359': 'Approaches from 225-250 yards',
 '02358': 'Approaches from 200-225 yards',
 '076': 'Approaches from 50-75 yards',
 '075': 'Approaches from 75-100 yards

In [43]:
def get_links(stat_ids, year):
    """
    Create a list of links for each statistic in a specified year.
    """
    base_url = 'https://www.pgatour.com/content/pgatour/stats/stat.'
    urls = [base_url + key + '.y' + str(year) + '.html' for key in stat_dict.keys()]
    return urls

In [44]:
def get_link(stat_id, year):
    """
    Create one specific link in a specific year
    """
    
    url = 'https://www.pgatour.com/content/pgatour/stats/stat.' + stat_id + '.y' + str(year) + '.html'
    return url

Now what do I need? I know I can access all of the meaning headers I want. I need to change some of the headers, for example headers that just say '%', for convenience, then I can rework my existing scraping functions to use the headers I have here. This will also make my make_data_frame function much cleaner and simpler to use as I won't have many separate cases involved.

In [45]:
test_headers = ['RANK THIS WEEK', 'RANK LAST WEEK', 'PLAYER NAME', 'SCORING AVERAGE (ACTUAL)', 'TOTAL STROKES', 'TOTAL ROUNDS']

test_headers[3] == 'SCORING AVERAGE (ACTUAL)' and test_headers[4] == 'TOTAL STROKES'


True

In [46]:
def get_stats(link):
    '''
    Collect the stats from a soup object and return as a list.
    '''
    response = req.get(link)
    soup = BeautifulSoup(response.content, 'lxml')
    table = soup.find('table', attrs = {'id': 'statsTable'})

    stats = []
    body = table.find('tbody')
    all_rows = body.findAll('tr')
    for row in all_rows:
        line = []
        for column in row.findAll('td'):
            data = column
            data = data.text
            line.append(data)
        stats.append(line)
    return stats

We are going to focus on a few specific stat categories for simplicity

In [47]:
#data = get_stats(links[0])

Now I can just use this logic to make my entire database!

In [48]:
def make_data_frames(links, year):
    """
    This function takes a scoring category and a provided url_list,
    calls the get_datas function to scrape the data, and creates a
    data frame for every url in the url list and appropriately
    renames the columns based upon the category provided. This returns
    a list of data frames.
    """
    headers_list = get_headers(links)
    df_list = []
    for i in range(len(links)):
        data = get_stats(links[i])
        df = pd.DataFrame(data, columns = headers_list[i])
        df['RANK THIS WEEK'] = df['RANK THIS WEEK'].str.replace('\n', '')
        df['RANK LAST WEEK'] = df['RANK LAST WEEK'].str.replace('\n', '')
        df['PLAYER NAME'] = df['PLAYER NAME'].str.replace('\n', '').str.strip()
        df_list.append(df)
    return df_list

In [62]:
def get_headers(links):
    headers_nested_list = []
    stat_dict_keys = [key for key in stat_dict.keys()]

    for i in range(len(links)):
        headers = []
        stat_name = stat_dict[stat_dict_keys[i]].upper()
        response = req.get(links[i])
        soup = BeautifulSoup(response.content, 'lxml')

            #Get rounds header
        rank_this_week = soup.find_all(class_="hidden-small hidden-medium")[0].get_text()
        headers.append('RANK ' + rank_this_week)

        player_name = soup.find_all(class_ = 'player-name')[0].get_text()
        headers.append(player_name)

            #Get other headers
        stat_headers = soup.find_all(class_="col-stat")
        for header in stat_headers:
                headers.append(header.get_text())

        headers.insert(1, 'RANK LAST WEEK')
        # All edge cases:

        if len(headers) > 4 and headers[4] == 'TOTAL PUTTS':
            if len(headers) == 7:
                        #headers_nested_list.append(headers)
                headers.insert(3, 'ROUNDS')
                headers[4] = stat_name
                #print('edge case 1')
                #print(headers)
                headers_nested_list.append(headers)
                #print(links[i])
                #print(stat_name)
                continue
            else:
                        #headers_nested_list.append(headers)
                headers[3] = stat_name
                #print('edge case 2')
                #print(headers)
                headers_nested_list.append(headers)
                #print(links[i])
                #print(stat_name)
                continue
                
        streak_stats = ['CONSECUTIVE CUTS', 'YTD PAR OR BETTER STREAK',
                       'BEST YTD 1-PUTT OR BETTER STREAK','BEST YTD STREAK W/O A 3-PUTT',
                       'CONSECUTIVE SAND SAVES', 'CONSECUTIVE FAIRWAYS HIT',
                       'CONSECUTIVE GIR', 'CONSECUTIVE HOLES BELOW PAR',
                       'CONSECUTIVE PAR 3 BIRDIES', 'CONSECUTIVE BIRDIES STREAK',
                       'CONSECUTIVE BIRDIES/EAGLES STREAK',
                       'CURRENT STREAK WITHOUT A 3-PUTT']

        if stat_name in streak_stats:
            headers[3] = stat_name
            #print('edge case 3')
            #print(headers)
            headers_nested_list.append(headers)
            #print(links[i])
            #print(stat_name)
            continue
            
        elif len(headers) > 6 and headers[5] == 'TOTAL ADJUSTMENT':
            #another edge case
            headers.insert(3, 'ROUNDS')
            headers[4] = stat_name
            #print('edge case 4')
            #print(stat_name)
            #print(headers)
            headers_nested_list.append(headers)
        elif len(headers) == 6 and headers[5] == 'TOTAL ROUNDS':
            if headers[4] == 'TOTAL STROKES':
                if stat_name == 'SCORING AVERAGE (ACTUAL)':
                    #print('edge case 5')
                    headers.insert(3, 'ROUNDS')
                    headers[4] = stat_name
                    #print(stat_name)
                    #print(headers[4])
                    headers_nested_list.append(headers)
                    #print(headers)
                    #print(links[i])
                    #print(stat_name)
                    continue
            if headers[4] == '# OF BIRDIES':
                #print('edge case 6')
                headers.insert(3, 'ROUNDS')
                headers[4] = stat_name
                #print(stat_name)
                #print(headers[4])
                headers_nested_list.append(headers)
                #print(headers)
                #print(links[i])
                #print(stat_name)
                continue
            #print(headers[3])
            headers[3] = stat_name
            #print(headers[3])
            #print('edge case 7')
            #print(stat_name)
            #print(headers)
            headers_nested_list.append(headers)

        else:
            #print(headers)
            if len(headers) == 3:
                headers.insert(3, 'ROUNDS')
                headers.insert(4, stat_name)
                headers_nested_list.append(headers)
                #print("edge case 8")
            
            else:
                headers.insert(3, 'ROUNDS')
                headers[4] = stat_name
                headers_nested_list.append(headers)
                #print("edge case 9")

        #print(headers)
        #print(links[i])
        #print(stat_name)
    return headers_nested_list

In [63]:
years = [str(i) for i in range(2010, 2022)]

In [64]:
for year in years:
    print(year)
    links = get_links(stat_ids, year)
    data_frames = make_data_frames(links, year)

    duplicate_columns = ['ROUNDS','ATTEMTPS','RELATIVE TO PAR',
                        'MEASURED ROUNDS','TOTAL DISTANCE (FEET)',
                        '# OF ATTEMPTS', 'SUCCESSES','TOTAL PUTTS',
                        'TOTAL ROUNDS','PUTTS MADE','TOTAL HOLES',
                        'ROUND','HOLE','CURRENT STREAK',
                        'RANK THIS WEEK','RANK LAST WEEK',
                        'ATTEMPTS', 'TOTAL STROKES']
    columns_to_drop = []
    df_one = data_frames[0]
    for column in df_one.columns:
        if column in duplicate_columns:
            columns_to_drop.append(column)
    df_one = df_one.drop(columns_to_drop, axis = 1)
        #Merge all Dataframes together

    for i in range(1, len(data_frames)):
        columns_to_drop = []
        for column in data_frames[i].columns:
            if column in duplicate_columns:
                columns_to_drop.append(column)
        df_one = pd.merge(df_one, data_frames[i].drop(columns_to_drop, axis = 1), on ='PLAYER NAME', how = 'left')

        #Only get people whose scoring average isn't null.
    #df_one = df_one.loc[df_one['SCORING'].isnull() == False]  

        #Add year column
    df_one['YEAR'] = year
    print(df_one.shape())

        #Concat dataframe to overall dataframe    
    if year == '2010':
        df_total = pd.DataFrame()
        df_total = pd.concat([df_total, df_one], axis=0)
    else:
        df_total = pd.concat([df_total, df_one], axis=0)


print('Finished Constructing Table')
#df_total

2010


C:\Users\zytow\AppData\Local\Temp/ipykernel_23424/2139410900.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_one['YEAR'] = year


2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
Finished Constructing Table


In [68]:
df_total

,PLAYER NAME,SG: TEE-TO-GREEN,SG:OTT,SG:APR,SG:ARG,SG: OFF-THE-TEE,TOTAL SG:OTT,DRIVING DISTANCE,TOTAL DISTANCE_x,TOTAL DRIVES,...,CONSECUTIVE SAND SAVES,YEAR/TOURN#_y,CONSECUTIVE FAIRWAYS HIT,CONSECUTIVE GIR,YEAR/TOURN#,CONSECUTIVE HOLES BELOW PAR,CONSECUTIVE PAR 3 BIRDIES,CONSECUTIVE BIRDIES STREAK,CONSECUTIVE BIRDIES/EAGLES STREAK,YEAR
0,Adam Scott,1.609,.472,1.164,-.027,.472,24.523,294.4,"40,045",136,...,4,2010390,19,15,2010150,3,4,3,3,2010
1,Vijay Singh,1.494,.369,.979,.145,.369,21.050,292.3,"43,267",148,...,6,2010390,14,17,2010400,4,2,4,3,2010
2,Steve Stricker,1.383,.191,.773,.419,.191,10.669,282.9,"40,735",144,...,8,2010360,24,11,2010350,4,2,4,2,2010
3,Jim Furyk,1.159,.150,.641,.367,.150,9.018,276.0,"40,853",148,...,9,2010410,19,18,2010360,4,3,4,3,2010
4,Phil Mickelson,1.151,.185,.738,.228,.185,10.556,299.1,"45,459",152,...,6,2010040,12,18,2010060,6,2,6,3,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,Sung Kang,-1.562,-.623,-.703,-.237,-.623,-44.211,291.2,"47,760",164,...,5,2021310,12,15,2021290,4,2,4,2,2021
202,Sebastian Cappelen,-1.624,-.186,-.831,-.607,-.186,-6.697,298.9,"31,083",104,...,5,2021200,13,11,2021170,3,1,3,1,2021
203,Rhein Gibson,-1.798,-.840,-.726,-.231,-.840,-30.250,290.2,"31,339",108,...,5,2021270,8,15,2021330,3,3,3,2,2021
204,Martin Trainer,-2.057,-.566,-1.459,-.032,-.566,-26.058,302.7,"34,508",114,...,4,2021310,10,14,2021430,3,2,3,2,2021


In [69]:
conn = db.connect('pga_database.db')
df_total.to_sql('pga_stats_table', conn, if_exists = 'replace')
conn.close()

c:\users\zytow\appdata\local\programs\python\python39\lib\site-packages\pandas\core\generic.py:2869: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(
c:\users\zytow\appdata\local\programs\python\python39\lib\site-packages\pandas\io\sql.py:900: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  temp.reset_index(inplace=True)


Now I know the make_data_frame function works. Now I can make my master dataframe

In [70]:
df_total.columns

Index(['PLAYER NAME', 'SG: TEE-TO-GREEN', 'SG:OTT', 'SG:APR', 'SG:ARG',
       'SG: OFF-THE-TEE', 'TOTAL SG:OTT', 'DRIVING DISTANCE',
       'TOTAL DISTANCE_x', 'TOTAL DRIVES',
       ...
       'CONSECUTIVE SAND SAVES', 'YEAR/TOURN#_y', 'CONSECUTIVE FAIRWAYS HIT',
       'CONSECUTIVE GIR', 'YEAR/TOURN#', 'CONSECUTIVE HOLES BELOW PAR',
       'CONSECUTIVE PAR 3 BIRDIES', 'CONSECUTIVE BIRDIES STREAK',
       'CONSECUTIVE BIRDIES/EAGLES STREAK', 'YEAR'],
      dtype='object', length=218)

In [72]:
df_total[df_total['PLAYER NAME'] == 'Adam Scott']

,PLAYER NAME,SG: TEE-TO-GREEN,SG:OTT,SG:APR,SG:ARG,SG: OFF-THE-TEE,TOTAL SG:OTT,DRIVING DISTANCE,TOTAL DISTANCE_x,TOTAL DRIVES,...,CONSECUTIVE SAND SAVES,YEAR/TOURN#_y,CONSECUTIVE FAIRWAYS HIT,CONSECUTIVE GIR,YEAR/TOURN#,CONSECUTIVE HOLES BELOW PAR,CONSECUTIVE PAR 3 BIRDIES,CONSECUTIVE BIRDIES STREAK,CONSECUTIVE BIRDIES/EAGLES STREAK,YEAR
0,Adam Scott,1.609,.472,1.164,-.027,.472,24.523,294.4,"40,045",136,...,4,2010390,19,15,2010150,3,4,3,3,2010
19,Adam Scott,.882,.793,.102,-.013,.793,35.670,299.7,"35,367",118,...,7,2011310,12,17,2011380,4,2,4,3,2011
6,Adam Scott,1.302,.605,.494,.203,.605,24.786,304.6,"35,335",116,...,6,2012280,13,17,2012160,4,3,4,2,2012
4,Adam Scott,1.337,.734,.548,.055,.734,32.289,297.8,"35,733",120,...,5,2013070,12,20,2013160,5,2,5,2,2013
5,Adam Scott,1.500,.725,.610,.166,.725,40.578,303.5,"41,275",136,...,6,2014110,18,19,2014470,4,2,4,3,2014
36,Adam Scott,.601,.682,.114,-.195,.682,21.818,311.6,"27,425",88,...,4,2015060,17,23,2015390,3,2,3,1,2015
0,Adam Scott,2.109,.576,1.490,.043,.576,29.932,304.5,"38,975",128,...,5,2016420,22,28,2016480,4,2,4,3,2016
17,Adam Scott,1.103,.524,.536,.043,.524,22.005,307.0,"30,697",100,...,8,2017040,10,15,2017040,3,3,3,2,2017
16,Adam Scott,1.064,.300,.639,.125,.300,17.987,304.6,"42,649",140,...,6,2018310,19,23,2018320,3,3,3,3,2018
3,Adam Scott,1.483,.285,.846,.351,.285,12.557,299.3,"31,731",106,...,7,2019030,12,19,2019470,4,2,4,3,2019


In [74]:
df_total.loc[df_total['PLAYER NAME'] == 'Adam Scott']

,PLAYER NAME,SG: TEE-TO-GREEN,SG:OTT,SG:APR,SG:ARG,SG: OFF-THE-TEE,TOTAL SG:OTT,DRIVING DISTANCE,TOTAL DISTANCE_x,TOTAL DRIVES,...,CONSECUTIVE SAND SAVES,YEAR/TOURN#_y,CONSECUTIVE FAIRWAYS HIT,CONSECUTIVE GIR,YEAR/TOURN#,CONSECUTIVE HOLES BELOW PAR,CONSECUTIVE PAR 3 BIRDIES,CONSECUTIVE BIRDIES STREAK,CONSECUTIVE BIRDIES/EAGLES STREAK,YEAR
0,Adam Scott,1.609,.472,1.164,-.027,.472,24.523,294.4,"40,045",136,...,4,2010390,19,15,2010150,3,4,3,3,2010
19,Adam Scott,.882,.793,.102,-.013,.793,35.670,299.7,"35,367",118,...,7,2011310,12,17,2011380,4,2,4,3,2011
6,Adam Scott,1.302,.605,.494,.203,.605,24.786,304.6,"35,335",116,...,6,2012280,13,17,2012160,4,3,4,2,2012
4,Adam Scott,1.337,.734,.548,.055,.734,32.289,297.8,"35,733",120,...,5,2013070,12,20,2013160,5,2,5,2,2013
5,Adam Scott,1.500,.725,.610,.166,.725,40.578,303.5,"41,275",136,...,6,2014110,18,19,2014470,4,2,4,3,2014
36,Adam Scott,.601,.682,.114,-.195,.682,21.818,311.6,"27,425",88,...,4,2015060,17,23,2015390,3,2,3,1,2015
0,Adam Scott,2.109,.576,1.490,.043,.576,29.932,304.5,"38,975",128,...,5,2016420,22,28,2016480,4,2,4,3,2016
17,Adam Scott,1.103,.524,.536,.043,.524,22.005,307.0,"30,697",100,...,8,2017040,10,15,2017040,3,3,3,2,2017
16,Adam Scott,1.064,.300,.639,.125,.300,17.987,304.6,"42,649",140,...,6,2018310,19,23,2018320,3,3,3,3,2018
3,Adam Scott,1.483,.285,.846,.351,.285,12.557,299.3,"31,731",106,...,7,2019030,12,19,2019470,4,2,4,3,2019
